Qiskit Project. Amplitude amplification.

First import all the packages we will need

In [166]:
import numpy as np
import math
import qiskit
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit import BasicAer
from qiskit.visualization import plot_histogram
from qiskit.circuit.library import MCXGate
from qiskit.extensions import UnitaryGate
from qiskit import execute

Now we characterize the system we will work with. Let us assume we have an unstructured list and we want to find a marked item. We need to specify the length of the list, set by number of qubits and how many marked items there are (which for now will be just 1).

In the standard Grover search, we have no additional information on the unstructured list. Here, we will assume that we have used a quantum algorithm that returns the marked item with probability p, which we also specify and will be used to build the initial quantum state. Here we assume the first algorithm is able to triple the probability to find the marked item compared to a completely random choice. The goal of this code is to further amplify this amplitude, i.e., to implement the amplitude amplification algorithm.

In [258]:
n_qubits = 8
n_good = 1
n_bad = 2**n_qubits - n_good
p_good = 3/2**n_qubits

We determine the marked item randomly and build the initial state. Of course, in a real scenario the marked item will unknown and the initial state, the output of the first algorithm, simply given. Here, however, we simply want to illustrate how amplification works.

In [259]:
good_numbers = np.random.randint(0,2**n_qubits-1,n_good)

good_states = []
for number in good_numbers:
    good_states.append(np.binary_repr(number,n_qubits))

In [260]:
initial_state = []

for number in range(2**n_qubits):
    if number in good_numbers:
        initial_state.append(np.sqrt(p_good/n_good))
    else:
        initial_state.append(np.sqrt((1-p_good)/n_bad))

Next, we build the functions required to build our circuit:

- Oracle. This function takes the solution as an int number and builds a circuit that marks the corresponding computational state with a (-) phase using the phase kickback trick. This is equivalent to reflecting through thee non-solution states. Returns a QuantumCircuit object.

- Reflection. This function takes the initial state and builds a circuit that performs a reflection through it. returns a QuantumCircuit object.

- Grover iteration. This function composes one oracle and reflection steps. Returns a QuantumCircuit.

In [261]:
def oracle(solution, n_qubits):
    
    solution_bin = np.binary_repr(solution,n_qubits)
    
    qr = QuantumRegister(n_qubits, 'q')
    anc = QuantumRegister(1, 'anc')
    
    oracle = QuantumCircuit(qr, anc)

    for i in range(n_qubits):
        if solution_bin[-i-1] == '0':
            oracle.x(qr[i])
    
    oracle.x(anc[0])
    oracle.h(anc[0])
    
    oracle.mcx(qr,anc)
    
    for i in range(n_qubits):
        if solution_bin[-i-1] == '0':
            oracle.x(qr[i])
    
    oracle.h(anc[0])
    oracle.x(anc[0])
    
    return oracle

In [262]:
def reflection(initial_state, n_qubits):
    initial_projector = np.tensordot(initial_state, initial_state, axes = 0)
    unitary = 2*initial_projector - np.identity(2**n_qubits)
    gate = UnitaryGate(unitary,'reflection_gate')
    reflection = QuantumCircuit(n_qubits)
    reflection.append(gate,list(range(n_qubits)))
    return reflection

In [263]:
def grover_iteration(oracle, reflection, n_qubits):
    
    grover = QuantumCircuit(n_qubits+1)
    
    qr = QuantumRegister(n_qubits, 'q')
    anc = QuantumRegister(1, 'anc')
    
    qc = QuantumCircuit(qr, anc)
    qc.compose(oracle,range(n_qubits+1),inplace=True)
    qc.compose(reflection,range(n_qubits),inplace=True)
    
    return qc

Now we build the circuit that performs the amplitude amplification itself. This is achieved by computing the number of required iterations using the num_iter function and then applying as many modified Grover iterations as required.

In [264]:
def num_iter(p_good):
    theta = np.arcsin(np.sqrt(p_good))
    return int(np.round(math.pi/(4 * theta) - 1/2))

In [265]:
qr = QuantumRegister(n_qubits, 'q')
anc = QuantumRegister(1, 'anc')
c = ClassicalRegister(n_qubits)
qc = QuantumCircuit(qr, anc,c)

qc.initialize(initial_state,qr)

n_iter = num_iter(p_good)

for iter in range(n_iter):
    qc.compose(grover_iteration(oracle(good_numbers[0],n_qubits),reflection(initial_state,n_qubits),n_qubits),range(n_qubits+1),inplace=True)

qc.measure(qr,c)

We check now that the algorithm works by executing it on a a simulator

In [266]:
backend = BasicAer.get_backend('qasm_simulator')
result = execute(qc, backend,shots=1024).result()
result.get_counts()

{'11010101': 1020, '00111001': 1, '01010000': 1, '11011100': 1, '11010000': 1}

And compare the probabilities

In [267]:
p_new_estimate = result.get_counts()[np.binary_repr(good_numbers[0],n_qubits)]/1024

In [268]:
p_new_estimate/p_good

85.0

In [269]:
p_new_estimate

0.99609375

There is a huge increase in the probability that we get the marked item upon measurement!